In [ ]:
import mqr
from mqr.plot import Figure
from mqr.nbtools import hstack, vstack, grab_figure

# Datasets

In [ ]:
import numpy as np
import pandas as pd

# Random glue data
data = pd.read_csv(mqr.sample_data('anova-glue.csv'), index_col='Run')

---
# Basic Plots

In [ ]:
with Figure(8, 3, 1, 3) as (fig, axs):
    mqr.plot.tools.sharey(fig, axs, True)
    mqr.plot.anova.main_effects(data, 'adhesion_force', factors=['primer', 'glue'], axs=axs[:2])
    mqr.plot.anova.interactions(data, 'adhesion_force', group='primer', factors=['glue'], axs=axs[2, None])
    axs[2].set_ylabel(None)
    [ax.grid(axis='y') for ax in axs]

---
# ANOVA

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
# Simple one factor
model = ols('adhesion_force ~ C(primer) + C(glue)', data=data)
result = model.fit()

vstack(
    mqr.anova.adequacy(result),
    mqr.anova.summary(result),
)

In [ ]:
with Figure(7, 3, 1, 2) as (fig, axs):
    mqr.plot.tools.sharey(fig, axs, remove_space=True)
    mqr.plot.anova.model_means(
        result,
        response='adhesion_force',
        factors=['primer', 'glue'],
        axs=axs)

In [ ]:
groups_primer = mqr.anova.groups(result, value='adhesion_force', factor='primer')
groups_glue = mqr.anova.groups(result, value='adhesion_force', factor='glue')

with Figure(7, 2, 1, 2) as (fig, axs):
    mqr.plot.tools.sharey(fig, axs)
    mqr.plot.anova.groups(groups_glue, ax=axs[0])
    mqr.plot.anova.groups(groups_primer, ax=axs[1])
    plot = grab_figure(fig)

vstack(
    plot,
    hstack(
        groups_glue,
        groups_primer,
    ),
)

---
## Residual analysis

In [ ]:
# See https://en.wikipedia.org/wiki/Studentized_residual#Internal_and_external_studentization

influence = result.get_influence()
N = influence.nobs
with Figure(6, 3) as (fig, ax):
    ax.plot(influence.resid_studentized_internal, linewidth=0, marker='o', fillstyle='none')
    ax.plot(influence.resid_studentized_external, linewidth=0, marker='x')
    ax.set_ylabel('residuals')

    axt = ax.twinx()
    axt.bar(range(N), 1-influence.cooks_distance[1], alpha=0.5)
    axt.set_ylim(0.0, 1.0)
    axt.set_ylabel("Cook's distance (1-p)")

    ax.legend(['stud. internal', 'stud. external'])
    ax.grid(axis='y')

In [ ]:
# NB: Factor plots are added manually. See how a 2*2 slice of the axes `ax[:2, :2]` is passed to the `residuals` function.
#     Any slice with 4 elements will work. The remaining factor plots are added manually into ax[2, 0] and ax[2, 1].

influence = result.get_influence()
resid = influence.resid_studentized_external

with Figure(7, 5, 3, 2, height_ratios=[3, 2, 2]) as (fig, ax):
    mqr.plot.regression.residuals(resid, result.fittedvalues, axs=ax[:2, :2])
    mqr.plot.regression.res_v_factor(resid, data['primer'], ax[2, 0])
    mqr.plot.regression.res_v_factor(resid, data['glue'], ax[2, 1])
    mqr.plot.regression.influence(result, 'cooks_dist', ax[1, 0])

    plot = grab_figure(fig)

vstack(
    plot,
    mqr.inference.dist.test_1sample(resid)
)